In [ ]:
import nltk
import json
import os
import pandas as pd
data_published=[]
data_text=[]
site=[]
news_dir='D:/ASU 2-2/SWM/CSE573-Directional-stock-prediction-master/CODE/News/'

class SentenceExtraction:
    def readData(self):
        for folder in os.scandir(news_dir):
            print(os.scandir(news_dir),type(os.scandir(news_dir)))
            if folder.name!=".DS_Store":
                for entry in os.scandir(news_dir+folder.name):

                    data = json.load(open(news_dir+folder.name+"/"+entry.name, encoding='utf-8'))
                    if data['published']!="":
                        data_published.append(data['published'])
                    else:
                        data_published.append("No Value")
                    if data['text']!="":
                        data_text.append(data['text'])
                    else:
                        data_text.append("No Value")
                    if data['thread']['site_full'] != '':
                        site.append(data['thread']['site_full'])
                    else:
                        site.append("unknown")
        # print(site)
        
    def extractSentence(self):
        df=pd.DataFrame({
                'time':data_published,
                'text':data_text,
                'source':site
            })
        from datetime import datetime

        
        day=[]
        time=[]
        source=[]
        for i in range(len(df)):
            day.append(pd.Timestamp(df.loc[i]['time']).date())
            time.append(pd.Timestamp(df.loc[i]['time'],tz="UTC").time())
            source.append(df.loc[i]['source'])
        df['day']=day
        df['24_time']=time
        df['published_source']=source
        df.drop(['time'],axis=1,inplace=True)

        my_df_appl  = pd.DataFrame(columns = ['text', 'day', '24_time','published_source'])
        my_df_amz  = pd.DataFrame(columns = ['text', 'day', '24_time','published_source'])
        
        
        key_words_appl=['apple','aapl']
        key_words_amz=['amazon','amzn']
        print(df.shape)
        for index, row in df.iterrows():
            print(index)
            paragraphs = nltk.sent_tokenize(row['text'].lower())
            sentences=[]
            for para in paragraphs:
                sentences_list = para.split("\n")
                for sentence in sentences_list:
                    if any(key in sentence for key in key_words_appl):
                        my_df_appl.loc[len(my_df_appl)] =[sentence,row['day'],row['24_time'],row['published_source']]
                    if any(key in sentence for key in key_words_amz):
                        my_df_amz.loc[len(my_df_amz)] =[sentence,row['day'],row['24_time'],row['published_source']]

#         print(my_df_appl)
#         print(my_df_amz)
        my_df_appl.sort_values(["day","24_time","published_source"], axis = 0, ascending = (True, True, True), 
                         inplace = True)
        my_df_amz.sort_values(["day","24_time","published_source"], axis = 0, ascending = (True, True, True), 
                         inplace = True)

        my_df_appl.to_csv('D:/ASU 2-2/SWM/CSE573-Directional-stock-prediction-master/CODE/ExtractedNewsSentenceWise2_appl.csv')
        my_df_amz.to_csv('D:/ASU 2-2/SWM/CSE573-Directional-stock-prediction-master/CODE/ExtractedNewsSentenceWise2_amz.csv')
        
s=SentenceExtraction()
s.readData()
print('Read Data')
s.extractSentence()